## AIS数据按每小时取平均，并且将所有数据合并到一个csv文件

In [ ]:
import pandas as pd
import numpy as np
import movingpandas as mdp
from datetime import datetime


In [27]:
# 读取船的AIS数据
shipnum = 1
df = pd.read_csv('processed_%d.csv' % shipnum,index_col=0)
df['postime'] = pd.to_datetime(df['postime'],format='ISO8601')

# 读取noon report，并且筛选出年月日
df_fuel = pd.read_csv('../raw_data/mrv.csv')
# 时间转换成datetime对象
df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date'],format='ISO8601')
# 先观察第一艘船
df_fuel = df_fuel[df_fuel['num'] == shipnum]
# 筛选出一直在跑的船，这边考虑daily_voyage_time_mrv大于等于24小时的轨迹
df_fuel = df_fuel[df_fuel['daily_voyage_time_mrv'] >= 24]
# 删除关键字段存在空值的行，daily_fuel_mrv
df_fuel = df_fuel.dropna(axis=0, subset = ['daily_fuel_mrv'])
# 序号重新排列
df_fuel = df_fuel.reset_index(drop=True)
# 将日期全部抽取出来
years = df_fuel['input_date'].dt.year
months = df_fuel['input_date'].dt.month
days = df_fuel['input_date'].dt.day

df_processed = pd.DataFrame(data=None,columns=['SOG','heading','draught', 'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction', 'postime', 'num'])

for i in range(1):
    for hours in range(24):
        # 同一个小时的所有点
        temp_hours = df[(df['postime'].dt.year ==years[i]) & (df['postime'].dt.month ==months[i]) & (df['postime'].dt.day ==days[i]) & (df['postime'].dt.hour == hours)]
        avg = temp_hours[['SOG','heading','draught',  'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction']].mean()
        avg['postime'] = datetime(years[i], months[i], days[i], hours, 0, 0)
        avg['num'] = shipnum
        avg = avg.to_frame(); avg = avg.transpose()
        df_processed = pd.concat([df_processed, avg], ignore_index = True)
    df_processed = df_processed.dropna(axis=0)

df_processed

,SOG,heading,draught,wind_val,wind_direction,wave_val,wave_direction,stream_val,stream_direction,postime,num
0,13.92,32.0,13.3,9.366,41.0,2.966,37.4,0.48,211.2,2021-01-04 00:00:00,1
1,13.95,31.5,13.3,9.71,40.0,2.99,37.0,0.51,206.0,2021-01-04 01:00:00,1
2,13.466667,32.333333,13.3,11.306667,36.0,2.923333,37.666667,0.646667,191.0,2021-01-04 02:00:00,1
3,13.3,37.666667,13.3,11.386667,36.666667,2.996667,37.666667,0.646667,184.333333,2021-01-04 03:00:00,1
4,13.533333,35.0,13.3,11.323333,36.666667,3.036667,37.333333,0.62,182.333333,2021-01-04 04:00:00,1
5,13.466667,37.0,13.3,11.53,41.0,3.05,39.0,0.58,179.0,2021-01-04 05:00:00,1
6,13.875,37.0,13.3,11.6875,39.5,3.14,38.25,0.745,182.75,2021-01-04 06:00:00,1
7,13.66,37.8,13.3,11.86,39.6,3.2,38.6,0.848,183.4,2021-01-04 07:00:00,1
8,11.4,39.2,13.3,10.92,36.0,3.18,37.0,1.33,196.0,2021-01-04 08:00:00,1
9,11.4,39.0,13.3,11.53,38.0,3.24,38.0,1.29,194.0,2021-01-04 09:00:00,1
